In [1]:
# The usual preamble
%matplotlib inline
#%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Make the graphs a bit prettier, and bigger
pd.set_option('display.mpl_style', 'default')
pd.set_option('display.max_columns', 100)


In [3]:
sfcrime = pd.read_csv('../data/train.csv',parse_dates='Dates',index_col='Dates')
sfcrime
sfcrime_mod = pd.read_csv('../data/train_mod.csv',parse_dates='Dates',index_col='Dates')
sfcrime_mod
sfcrime_test = pd.read_csv('../data/test.csv',parse_dates='Dates',index_col='Dates')
sfcrime_test
sfcrime_mod

,Category,Descript,PdDistrict,Resolution,Address,X,Y,Hour,Weekday,Year,Month,Day,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
Dates,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,NORTHERN,"ARREST, BOOKED",Other,-122.425892,37.774599,23,2,2015,5,133,0.098480,0.108200,0.133005,0.062284,0.159271,0.104630,0.114638,0.083582,0.085981,0.112350,0.093750,0.024440,0.121783,0.123209,0.068493,0.098676,0.163694,0.065160,0.157551,0.075263,0.110938,0.096947,0.227273,0.244655,0.086998,0.114783,0.045221,0.100050,0.099134,0.060811,0.154185,0.131890,0.104094,0.166667,0.106470,0.120827,0.116974,0.108945,0.092227,0.0,0.5,0.0,0.25,1.00,0.50,0.5,0.50,0.0,0.50
2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,NORTHERN,"ARREST, BOOKED",Other,-122.425892,37.774599,23,2,2015,5,133,0.098480,0.108200,0.133005,0.062284,0.159271,0.104630,0.114638,0.083582,0.085981,0.112350,0.093750,0.024440,0.121783,0.123209,0.068493,0.098676,0.163694,0.065160,0.157551,0.075263,0.110938,0.096947,0.227273,0.244655,0.086998,0.114783,0.045221,0.100050,0.099134,0.060811,0.154185,0.131890,0.104094,0.166667,0.106470,0.120827,0.116974,0.108945,0.092227,0.0,0.5,0.0,0.25,1.00,0.50,0.5,0.50,0.0,0.50
2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,NORTHERN,"ARREST, BOOKED",Other,-122.424363,37.800414,23,2,2015,5,133,0.098480,0.108200,0.133005,0.062284,0.159271,0.104630,0.114638,0.083582,0.085981,0.112350,0.093750,0.024440,0.121783,0.123209,0.068493,0.098676,0.163694,0.065160,0.157551,0.075263,0.110938,0.096947,0.227273,0.244655,0.086998,0.114783,0.045221,0.100050,0.099134,0.060811,0.154185,0.131890,0.104094,0.166667,0.106470,0.120827,0.116974,0.108945,0.092227,0.0,0.5,0.0,0.25,1.00,0.50,0.5,0.50,0.0,0.50
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,23,2,2015,5,133,0.098480,0.108200,0.133005,0.062284,0.159271,0.104630,0.114638,0.083582,0.085981,0.112350,0.093750,0.024440,0.121783,0.123209,0.068493,0.098676,0.163694,0.065160,0.157551,0.075263,0.110938,0.096947,0.227273,0.244655,0.086998,0.114783,0.045221,0.100050,0.099134,0.060811,0.154185,0.131890,0.104094,0.166667,0.106470,0.120827,0.116974,0.108945,0.092227,0.0,0.5,0.0,0.25,1.00,0.50,0.5,0.50,0.0,0.50
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,PARK,NONE,Other,-122.438738,37.771541,23,2,2015,5,133,0.042961,0.045723,0.036946,0.024221,0.078574,0.062731,0.077601,0.047674,0.087383,0.042882,0.031250,0.052953,0.048826,0.058397,0.006849,0.040154,0.052293,0.078297,0.018776,0.104390,0.064190,0.049009,NaN,0.000668,0.037285,0.041609,0.249229,0.044467,0.048314,0.040541,0.038546,0.039370,0.052715,NaN,0.039585,0.058424,0.073688,0.054911,0.041730,0.0,0.0,0.5,0.50,0.50,1.00,0.5,0.25,0.5,0.25
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,INGLESIDE,NONE,Other,-122.403252,37.713431,23,2,2015,5,133,0.120291,0.110997,0.078818,0.183391,0.090627,0.039583,0.090829,0.043968,0.044393,0.066895,0.113281,0.144603,0.092092,0.072726,0.109589,0.148654,0.058525,0.055702,0.021224,0.122398,0.074244,0.104635,NaN,0.003474,0.210962,0.121217,0.076053,0.130195,0.114859,0.148649,0.076432,0.127953,0.100815,NaN,0.056784,0.120157,0.166602,0.059885,0.132086,0.5,0.0,1.0,0.50,0.00,0.50,0.0,0.00,0.5,0.00
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,INGLESIDE,NONE,Other,-122.423327,37.725138,23,2,2015,5,133,0.120291,0.110997,0.078818,0.1

In [4]:
nrows = 50000
from math import floor
step = int(floor( sfcrime.shape[0]/nrows))
import random
def shuffle(df):
    index = list(df.index)
    random.shuffle(index)
    df = df.ix[index]
    df.reset_index()
    return df
#apparently you should shuffle before training the SGD
sfcrime = shuffle(sfcrime)

#sample evenly over time
#sfcrime_short = sfcrime.iloc[::step,:].copy()
sfcrime_short = sfcrime[:].copy()
sfcrime_short['Hour'] = sfcrime_short.index.hour
sfcrime_short['Year'] = sfcrime_short.index.year
sfcrime_short['Month'] = sfcrime_short.index.month
sfcrime_short['Day'] = sfcrime_short.index.dayofyear


NewFrame = pd.concat([pd.get_dummies(sfcrime_short['DayOfWeek']),
                      sfcrime_short[['X','Y']],
                      sfcrime_short[['Hour','Month','Year','Day']],
                      pd.get_dummies(sfcrime_short['PdDistrict']) #,
                      #pd.get_dummies(sfcrime_short['Resolution'])
                     ],axis=1)
sfcrime_test_short = sfcrime_test[:].copy()
sfcrime_test_short['Hour'] = sfcrime_test_short.index.hour
sfcrime_test_short['Year'] = sfcrime_test_short.index.year
sfcrime_test_short['Month'] = sfcrime_test_short.index.month
sfcrime_test_short['Day'] = sfcrime_test_short.index.dayofyear

NewFrame_test = pd.concat([pd.get_dummies(sfcrime_test_short['DayOfWeek']),
                      sfcrime_test_short[['X','Y']],
                      sfcrime_test_short[['Hour','Month','Year','Day']],
                      pd.get_dummies(sfcrime_test_short['PdDistrict']) #,
                      #pd.get_dummies(sfcrime_test_short['Resolution'])
                     ],axis=1)
#NewFrame
#sfcrime_mod_short = sfcrime_mod.iloc[::step,:].copy()
sfcrime_mod_short = sfcrime_mod[:].copy()

NewFrame_mod = pd.concat([pd.get_dummies(sfcrime_mod_short['Resolution']),
                          #pd.get_dummies(sfcrime_mod_short['Address']),
                      sfcrime_mod_short.drop(['Resolution','Address','PdDistrict'],axis=1)
                     ],axis=1)
NewFrame_mod = NewFrame_mod.fillna(0.)

In [4]:

X = np.array(NewFrame)
y = np.array(sfcrime_short['Category'].astype('category'))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.shape,X_test.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_test_scaled_final = scaler.transform(np.array(NewFrame_test))

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
"""
#this is overwriting the above
X = np.array(NewFrame_mod.drop(['Category','Descript'],axis=1))
y = np.array(sfcrime_mod_short['Category'].astype('category'))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.shape,X_test.shape

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
"""

"\n#this is overwriting the above\nX = np.array(NewFrame_mod.drop(['Category','Descript'],axis=1))\ny = np.array(sfcrime_mod_short['Category'].astype('category'))\nfrom sklearn.cross_validation import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X,y)\nX_train.shape,X_test.shape\n\nfrom sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nscaler.fit(X_train)\nX_train_scaled = scaler.transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n"

In [5]:
from sklearn.linear_model import SGDClassifier

In [6]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_train_features = rbf_feature.fit_transform(X_train_scaled)
X_test_features = rbf_feature.transform(X_test_scaled)
X_test_features_final = rbf_feature.transform(X_test_scaled_final)

In [19]:
param_grid = {'alpha' : 10. ** np.arange(-4, -1)}
from sklearn.grid_search import GridSearchCV
grid_search = GridSearchCV(SGDClassifier(loss='log'), param_grid, verbose=3,scoring='log_loss')
grid_search.fit(X_train_features,y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-2.643933 - 7.5min
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-2.643749 - 6.3min
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-2.643922 - 5.9min
[CV] alpha=0.001 .....................................................
[CV] ........................... alpha=0.001, score=-2.650603 - 5.3min
[CV] alpha=0.001 .....................................................
[CV] ........................... alpha=0.001, score=-2.650509 - 5.1min
[CV] alpha=0.001 .....................................................
[CV] ........................... alpha=0.001, score=-2.650456 - 5.0min
[CV] alpha=0.01 ......................................................
[CV] .............

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 50.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001,  0.001 ,  0.01  ])},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=3)

In [20]:
grid_search.best_params_

{'alpha': 0.0001}

In [7]:
#I can probabily use the decision_functions score instead since I don't need a 
#probability and that apparently takes a lot of work
svm = SGDClassifier(loss='log')

In [8]:
%%time
#svm.fit(X_train_scaled, y_train)
svm.fit(X_train_features, y_train)


CPU times: user 6min 7s, sys: 10.4 s, total: 6min 17s
Wall time: 6min 27s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [21]:
#svm.score(X_train_scaled, y_train)
#svm.score(X_train_features, y_train)
grid_search.score(X_train_features, y_train)

-2.6432168999765104

In [22]:
#svm.score(X_test_scaled,y_test)
#svm.score(X_test_features,y_test)
grid_search.score(X_test_features,y_test)

-2.6438881112253521

In [23]:
#probs = svm.decision_function(X_test_scaled_final)
#probs = svm.predict_proba(X_test_features_final)
probs = grid_search.predict_proba(X_test_features_final)

probs

array([[ 0.00107744,  0.07679849,  0.00079803, ...,  0.084206  ,
         0.02695656,  0.0076473 ],
       [ 0.00105271,  0.07640138,  0.00079139, ...,  0.08878456,
         0.02551287,  0.00758565],
       [ 0.00105211,  0.07354279,  0.00084644, ...,  0.07754004,
         0.0348401 ,  0.00706933],
       ..., 
       [ 0.00101708,  0.09260961,  0.00085432, ...,  0.08354746,
         0.03548176,  0.00735007],
       [ 0.00112398,  0.08173609,  0.00085743, ...,  0.07359033,
         0.03982059,  0.00713092],
       [ 0.00103817,  0.07039868,  0.0008392 , ...,  0.09338547,
         0.03263281,  0.00782875]])

In [16]:
grid_search.best_estimator_.classes_

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], 
      dtype='|S27')

In [24]:
#output = pd.DataFrame(probs,columns=svm.classes_)
output = pd.DataFrame(probs,columns=grid_search.best_estimator_.classes_)
output.index.name = 'Id'
#output['BAD CHECKS'] = 0.
#output['PORNOGRAPHY/OBSCENE MAT'] = 0.
#output['RECOVERED VEHICLE'] = 0.
#output['TREA'] = 0.
output

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.001077,0.076798,0.000798,0.000368,0.047213,0.003563,0.001276,0.028073,0.003586,0.002527,0.000612,0.000514,0.012361,0.024846,0.000143,0.002860,0.262931,0.001310,0.000860,0.037553,0.095786,0.115544,0.000027,0.004550,0.001465,0.018036,0.002440,0.014218,0.006951,0.000349,0.005328,0.000431,0.034794,0.000012,0.006007,0.065981,0.084206,0.026957,0.007647
1,0.001053,0.076401,0.000791,0.000366,0.049950,0.003534,0.001237,0.022713,0.003566,0.002560,0.000607,0.000510,0.011472,0.025366,0.000142,0.002737,0.271821,0.001318,0.000843,0.035252,0.095133,0.115946,0.000027,0.004692,0.001410,0.016437,0.002459,0.013828,0.006578,0.000354,0.005306,0.000438,0.031781,0.000012,0.006001,0.065475,0.088785,0.025513,0.007586
2,0.001052,0.073543,0.000846,0.000381,0.042083,0.003737,0.001338,0.053953,0.003524,0.002547,0.000570,0.000540,0.020495,0.031108,0.000147,0.003330,0.230581,0.001474,0.000974,0.033773,0.104801,0.116608,0.000028,0.007195,0.001439,0.018571,0.002445,0.012534,0.008033,0.000352,0.005446,0.000464,0.037473,0.000012,0.006295,0.052859,0.077540,0.034840,0.007069
3,0.001034,0.080155,0.000877,0.000384,0.036113,0.004338,0.001371,0.038347,0.003612,0.005360,0.000642,0.000502,0.025765,0.028446,0.000149,0.003232,0.184873,0.001377,0.001112,0.032476,0.133706,0.125625,0.000028,0.006049,0.001492,0.018287,0.002474,0.015693,0.013564,0.000393,0.004810,0.000470,0.043467,0.000012,0.006139,0.054633,0.083413,0.031780,0.007797
4,0.001034,0.080155,0.000877,0.000384,0.036113,0.004338,0.001371,0.038347,0.003612,0.005360,0.000642,0.000502,0.025765,0.028446,0.000149,0.003232,0.184873,0.001377,0.001112,0.032476,0.133706,0.125625,0.000028,0.006049,0.001492,0.018287,0.002474,0.015693,0.013564,0.000393,0.004810,0.000470,0.043467,0.000012,0.006139,0.054633,0.083413,0.031780,0.007797
5,0.001115,0.074680,0.000775,0.000393,0.041955,0.004269,0.001301,0.077791,0.003371,0.002231,0.000560,0.000547,0.013698,0.026389,0.000145,0.003909,0.196561,0.001470,0.001000,0.037905,0.083110,0.139436,0.000028,0.007680,0.001531,0.020727,0.002955,0.012958,0.006258,0.000350,0.005529,0.000478,0.031701,0.000012,0.005756,0.055086,0.088014,0.040132,0.008194
6,0.001017,0.080253,0.000903,0.000378,0.039581,0.004368,0.001351,0.039516,0.003560,0.005282,0.000667,0.000497,0.026062,0.029107,0.000148,0.003129,0.189634,0.001358,0.001117,0.031841,0.130688,0.122988,0.000028,0.004591,0.001500,0.018353,0.002232,0.014416,0.013066,0.000380,0.004967,0.000469,0.041959,0.000012,0.005799,0.050231,0.086516,0.033774,0.008265
7,0.001033,0.082885,0.000935,0.000378,0.044746,0.004659,0.001333,0.041484,0.003466,0.004632,0.000686,0.000493,0.022993,0.029237,0.000146,0.003245,0.200365,0.001341,0.001126,0.031810,0.121000,0.123599,0.000028,0.004126,0.001493,0.019239,0.002130,0.013690,0.012433,0.000378,0.005344,0.000467,0.036736,0.000012,0.006082,0.048136,0.083143,0.036452,0.008518
8,0.001092,0.073754,0.000819,0.000376,0.051368,0.004086,0.001253,0.042675,0.003302,0.003048,0.000602,0.000499,0.023904,0.034476,0.000145,0.003588,0.256623,0.001331,0.001057,0.033277,0.100732,0.099958,0.000028,0.005130,0.001441,0.016788,0.002085,0.011228,0.008430,0.000368,0.004760,0.000444,0.036105,0.000012,0.006361,0.052277,0.077709,0.032141,0.006729


In [25]:
output.to_csv('../data/submission.csv')